# Q-1. Take any YouTube videos link and your task is to extract the comments from that videos and store it in a csv file and then you need define what is most demanding topic in that videos comment section.

In [5]:
!pip install selenium

     ---------------------------------------- 6.6/6.6 MB 169.4 kB/s eta 0:00:00
     ------------------------------------ 384.9/384.9 kB 222.1 kB/s eta 0:00:00
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)


  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
     -------------------------------------- 58.3/58.3 kB 307.8 kB/s eta 0:00:00


In [6]:
!pip install beautifulsoup4

In [7]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [8]:
# Set the path to your ChromeDriver executable
webdriver_path = "path/to/chromedriver"

In [9]:
# Define the YouTube video URL
video_url = "https://youtu.be/cmtD1SgRqm8"

In [10]:
# Configure ChromeDriver options
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run Chrome in headless mode

In [11]:
# Create a new ChromeDriver instance
driver = webdriver.Chrome(service=Service(webdriver_path), options=options)

In [12]:
# Open the YouTube video page
driver.get(video_url)

In [13]:
# Scroll down to load comments dynamically
scroll_pause_time = 2  # Time to pause between each scroll (adjust as needed)
scroll_height = driver.execute_script("return document.documentElement.scrollHeight")

In [14]:
while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(scroll_pause_time)

    # Calculate the new scroll height and check if it has reached the bottom
    new_scroll_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_scroll_height == scroll_height:
        break
    scroll_height = new_scroll_height

In [15]:
# Extract the HTML source of the page
html_source = driver.page_source

In [16]:
# Close the ChromeDriver instance
driver.quit()

In [17]:
# Parse the HTML source with BeautifulSoup
soup = BeautifulSoup(html_source, "html.parser")

In [18]:
# Find and extract the comment elements
comment_elements = soup.find_all("yt-formatted-string", class_="style-scope ytd-comment-renderer")

In [19]:
# Extract the text content of the comments
comments = [comment.get_text(strip=True) for comment in comment_elements]

In [20]:
# Store comments in a CSV file
with open("youtube_comments.csv", "w", encoding="utf-8", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Comment"])
    writer.writerows(zip(comments))

print("Comments extracted and stored in youtube_comments.csv")

Comments extracted and stored in youtube_comments.csv


In [13]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [14]:
# Load the extracted comments from the CSV file
comments_df = pd.read_csv("youtube_comments.csv")

In [15]:
comments_df.head(5)

,Comment
0,Upcoming Job Guarantee ProgramsFull Stack Data...
1,sir i have doubtwhat is the difference between...
2,What to choose between this course and data sc...
3,the ineuron page shows classes are on weekends...
4,as you said you can become data analyst by doi...


In [16]:
comments_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Comment  13 non-null     object
dtypes: object(1)
memory usage: 232.0+ bytes


In [17]:
nltk.data.path.append('path/to/nltk_data') #https://www.nltk.org/data.html

In [30]:
def preprocess_text(comment):
    comment = comment.lower()
    comment = re.sub(r"[^\w\s]", "", comment)  # Remove special characters
    tokens = comment.split(" ")
    filtered_tokens = [token for token in tokens]
    return " ".join(filtered_tokens)

In [31]:
import re
comments_df["ProcessedComment"] = comments_df["Comment"].apply(preprocess_text)

In [32]:
# Create a document-term matrix
vectorizer = CountVectorizer()
dtm = vectorizer.fit_transform(comments_df["ProcessedComment"])

In [33]:
# Perform topic modeling using LDA
n_topics = 5  # Set the number of topics
lda_model = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda_model.fit(dtm)

LatentDirichletAllocation(n_components=5, random_state=42)

In [34]:
# Get the dominant topics for each comment
dominant_topics = lda_model.transform(dtm)
comments_df["DominantTopic"] = dominant_topics.argmax(axis=1)

In [35]:
# Calculate the topic distribution
topic_distribution = comments_df["DominantTopic"].value_counts(normalize=True)

In [36]:
# Determine the most demanding topic
most_demanding_topic = topic_distribution.idxmax()
print("The most demanding topic in the comment section is Topic", most_demanding_topic)

The most demanding topic in the comment section is Topic 0
